In [7]:
tacke_1 = [[958, 38, 1], [1117, 111, 1], [874, 285, 1], [707, 218, 1], [292, 569, 1], [770, 969, 1], [770, 1465, 1], [317, 1057, 1]]
tacke_2 = [[933, 33, 1], [1027, 132, 1], [692, 223, 1], [595, 123, 1], [272, 360, 1], [432, 814, 1], [414, 1284, 1], [258, 818, 1]]

In [25]:
import numpy as np
import numpy.linalg as LA

In [26]:
def jed(x, y):
  return [x[0]*y[0], x[1]*y[0], x[2]*y[0], x[0]*y[1], x[1]*y[1], x[2]*y[1], x[0]*y[2], x[1]*y[2], x[2]*y[2]]

In [34]:
matrix = np.zeros((8, 9))

for i in range(8):
  matrix[i] = jed(tacke_1[i], tacke_2[i])

In [35]:
u, d, v = LA.svd(matrix)

F = np.zeros((3, 3))

n = v.shape[1]
k = 0
for i in range(3):
  for j in range(3):
    F[i][j] = v[k][n-1]
    k += 1

print(F)

[[-7.84115037e-07  8.99762370e-07  3.43898904e-07]
 [ 8.26053935e-06 -1.95917969e-04 -1.70354384e-03]
 [ 3.54170315e-03  1.81927302e-03  9.99990603e-01]]


In [36]:
def test(x, y, F):
  x = np.array(x)
  y = np.array(y)
  return y @ F @ x

for i in range(8):
  if test(tacke_1[i], tacke_2[i], F) != 0:
    print(test(tacke_1[i], tacke_2[i], F))

print(LA.det(F))

3.7526932427664645
2.483940197482996
-6.904079119731003
-1.0568962553179984
-36.73035996633302
-145.1366306907186
-355.86554089015664
-164.42024950538675
1.3857331671975401e-10
